In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

import os

print(os.listdir("../input"))


def add_missing_cols(df, l_cols ):
    st_missing_cols = set(l_cols) - set(df.columns)
    for c in st_missing_cols:
        df[c] = 0

cfg_epoch            = 250
cfg_batch            = 50
cfg_validation_split = 0.05
cfg_path_tr = "../input/train/train.csv"
cfg_path_te  = "../input/test/test.csv"

l_cols = ['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 
                         'Color1', 'Color2', 'Color3', 'MaturitySize', 'FurLength',
                         'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'Quantity', 
                         'Fee', 'VideoAmt', 'PhotoAmt', 'State']
l_cols_dum = ["Type", "Breed1", "Breed2", "Color1", "Color2", 
                               "Color3", "Gender", "MaturitySize", "FurLength", "State", 
                               'Health', 'Sterilized', 'Dewormed', 'Vaccinated']

df_X_te = pd.read_csv(cfg_path_te, header=0, usecols=l_cols)
df_X_te = pd.get_dummies(df_X_te, columns=l_cols_dum)

# refine dummy columns with df_X_tr, df_X_te
df_X_tr = pd.read_csv(cfg_path_tr, header=0, usecols=l_cols)
df_X_tr = pd.get_dummies(df_X_tr, columns=l_cols_dum)
add_missing_cols(df_X_tr, df_X_te.columns)
add_missing_cols(df_X_te, df_X_tr.columns)
# normalize AdoptionSpeed in range of [0..1]
df_Y_tr = pd.read_csv(cfg_path_tr, header=0, usecols=['AdoptionSpeed'])
df_Y_tr = pd.get_dummies(df_Y_tr, columns=['AdoptionSpeed'])
#df_Y_tr = df_Y_tr['AdoptionSpeed'].apply(lambda v: v/4)

# build model
n_X   = df_X_tr.shape[1]
n_Y   = df_Y_tr.shape[1]
model = Sequential()
model.add(Dense(n_X, input_dim=n_X, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(n_X, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(n_Y, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(df_X_tr, df_Y_tr, epochs=cfg_epoch, batch_size=cfg_batch,
         shuffle=True, validation_split=cfg_validation_split, verbose=2)
#l_scores = model.evaluate(df_X_tr, df_Y_tr)
#print(f'{model.metrics_names[1]}: {l_scores[1]:.2f}')
l_pred = model.predict_classes(df_X_te)
l_id   = pd.read_csv(cfg_path_te)['PetID']
my_submission = pd.DataFrame({'PetID': l_id, 'AdoptionSpeed': l_pred})
my_submission.to_csv('submission.csv', index=False)